In [ ]:

import napari
import os
import glob
from pathlib import Path
from tifffile import imread
import numpy as np
import csv
import pandas as pd
from qtpy.QtWidgets import QComboBox, QPushButton, QSlider
%gui qt

In [ ]:
class EventViewer(object):
    
    def __init__(self, viewer, imagename, Name, csv_dir, save = False, newimage = True):
     
          self.save = save
          self.newimage = newimage
          self.viewer = viewer
           
          print('reading image')      
          self.imagename = imagename  
          self.image = imread(imagename)
          print('image read')
          self.Name = Name
            
          self.csv_dir = csv_dir
          
          
          self.click()
    def click(self):

        
                ClassOut = 'OutofFocus'
                ClassCad = 'CadFocus'
                ClassNuclei = 'NucleiFocus'
                '
                if self.save == True:
 
                        ClassOutdata = self.viewer.layers[ClassOut].data
                        bgdf = pd.DataFrame(ClassOutdata, index = None, columns = ['Z', 'Y', 'X'])
                        bgdf.to_csv(self.csv_dir + '/' + 'ONEAT' + ClassOut + self.Name +  '.csv', index = False, mode = 'w')

                        ClassCaddata = self.viewer.layers[ClassCad].data
                        caddf = pd.DataFrame(ClassCaddata, index = None, columns = ['Z', 'Y', 'X'])
                        caddf.to_csv(self.csv_dir + '/' +'ONEAT' + ClassCad + self.Name +  '.csv', index = False, mode = 'w')

                        ClassNucleidata = self.viewer.layers[ClassNuclei].data
                        nucdf = pd.DataFrame(ClassNucleidata, index = None, columns = ['Z', 'Y', 'X'])
                        nucdf.to_csv(self.csv_dir + '/' + 'ONEAT' + ClassNuclei + self.Name +  '.csv', index = False, mode = 'w')

                        
        
        
                if self.newimage == True:
                     for layer in list(self.viewer.layers):

                            self.viewer.layers.remove(layer) 
                    
                if self.save == False:
                        self.viewer.add_image(self.image, name = self.Name)


                        # add the first points layer for ClassOutdata T point
                        self.viewer.add_points(name= ClassOut, face_color='red', ndim = 3)

                        # add the second points layer for ClassCad T point
                        self.viewer.add_points(name= ClassCad, face_color='blue', ndim = 3)

                        # add the second points layer for ClassNuclei T point
                        self.viewer.add_points(name= ClassNuclei, face_color='green', ndim = 3)

             
                        
                        # programatically enter add mode for both Points layers to enable editing

                        self.viewer.layers[ClassOut].mode = 'add'
                        self.viewer.layers[ClassCad].mode = 'add'
                        self.viewer.layers[ClassNuclei].mode = 'add'
                        '

In [ ]:
sourcedir = '/home/Images/'
csv_dir = '/home/CsvFiles/'
Imageids = []
Boxname = 'ImageIDBox'
Path(csv_dir).mkdir(exist_ok = True)



Raw_path = os.path.join(sourcedir, '*tif')
X = glob.glob(Raw_path)
for imagename in X:
         Imageids.append(imagename)
         Name = os.path.basename(os.path.splitext(imagename)[0])

imageidbox = QComboBox()   
imageidbox.addItem(Boxname)   
tracksavebutton = QPushButton('Save Clicks')

for i in range(0, len(Imageids)):


     imageidbox.addItem(str(Imageids[i]))
        
        
viewer = napari.Viewer()        
viewer.window.add_dock_widget(imageidbox, name="Image", area='bottom')    
viewer.window.add_dock_widget(tracksavebutton, name="Save Clicks", area='bottom')
imageidbox.currentIndexChanged.connect(
         lambda trackid = imageidbox: EventViewer(
                 viewer,
                  imageidbox.currentText(),
                       os.path.basename(os.path.splitext(imageidbox.currentText())[0]), csv_dir, False, True ))     

tracksavebutton.clicked.connect(
        lambda trackid= tracksavebutton:EventViewer(
                 viewer,
                  imageidbox.currentText(),
                       os.path.basename(os.path.splitext(imageidbox.currentText())[0]), csv_dir, True, False ))